In [1]:
import pandas as pd
import numpy as np
import importlib
import gc
import sys
import warnings
sys.path.append('./scripts')  
import preprocesamiento
import feature_engineering
import model_lgb
importlib.reload(preprocesamiento)
importlib.reload(model_lgb)
importlib.reload(feature_engineering)
warnings.filterwarnings("ignore")

c:\Users\Usuario\.conda\envs\py311lab3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.


# Exp 2

-   Es el EXP 1 pero con linear_tree=True


##### Levantamos el dataset con target ya calculado

In [2]:
df = pd.read_csv("./datasets/periodo_x_producto_con_target.csv", sep=',', encoding='utf-8')
df.shape

(31362, 19)

##### Preprocesamiento a la minima expresión :)

In [3]:
##### aplicamos OHE
df = preprocesamiento.aplicarOHE(df)
df.shape

(31362, 162)

### Feature Engineering

##### Neural Prophet

In [4]:
neural_prophet_fe = pd.read_csv("./datasets/features_neuralprophet_completo.csv", sep=',', encoding='utf-8')
neural_prophet_fe['ds'] = pd.to_datetime(neural_prophet_fe['ds'], errors='coerce')
# Versión alternativa más robusta:
neural_prophet_fe['periodo'] = neural_prophet_fe['ds'].apply(
    lambda x: x.year * 100 + x.month if pd.notnull(x) else None
)
neural_prophet_fe = neural_prophet_fe[['periodo', 'product_id', 'trend', "season_yearly", "season_monthly"]]
df = df.merge(neural_prophet_fe, on=['periodo', 'product_id'], how='left')
df.shape

(31362, 165)

##### Prophet

In [5]:
prophet_fe = pd.read_csv("./datasets/prophet_features_tn_zscore.csv", sep=',', encoding='utf-8')
prophet_fe['ds'] = pd.to_datetime(prophet_fe['ds'], errors='coerce')
prophet_fe['periodo'] = prophet_fe['ds'].apply(
    lambda x: x.year * 100 + x.month if pd.notnull(x) else None
)
prophet_fe = prophet_fe[['periodo', 'product_id', 'trend_add', "yearly_add", "additive_terms", 'trend_mult', 'yearly_mult', 'multiplicative_terms']]
df = df.merge(prophet_fe, on=['periodo', 'product_id'], how='left')
df.shape

(31362, 171)

##### FE Moviles

In [6]:
df = feature_engineering.get_lags(df, "tn", 201912)
df = feature_engineering.get_delta_lags(df, "tn", 24)
df = feature_engineering.get_rolling_means(df, "tn", 201912)
df = feature_engineering.get_rolling_stds(df, "tn", 201912)
df = feature_engineering.get_rolling_mins(df, "tn", 201912)
df = feature_engineering.get_rolling_maxs(df, "tn", 201912)
# df = feature_engineering.get_rolling_medians(df, "tn", 201912)
# df = feature_engineering.get_rolling_skewness(df, "tn", 201912)
# df = feature_engineering.get_autocorrelaciones(df, "tn", 201912)
df.shape

(31362, 626)

##### FE Moviles sobre otras variables

In [7]:
# #  stock final
# df = feature_engineering.get_lagsEspecificos(df, col='stock_final_zscore')
# df = feature_engineering.get_delta_lags_especificos(df, col='stock_final_zscore')
# df = feature_engineering.get_rolling_means_especificos(df, col='stock_final_zscore')
# df = feature_engineering.get_rolling_stds_especificos(df, col='stock_final_zscore')
# df = feature_engineering.get_rolling_medians_especificos(df, col='stock_final_zscore')
# df = feature_engineering.get_rolling_mins_especificos(df, col='stock_final_zscore')
# df = feature_engineering.get_rolling_maxs_especificos(df, col='stock_final_zscore')
# df = feature_engineering.get_rolling_skewness_especificos(df, col='stock_final_zscore')

# #  cust_request_qty
# df = feature_engineering.get_lagsEspecificos(df, col='cust_request_qty_zscore')
# df = feature_engineering.get_delta_lags_especificos(df, col='cust_request_qty_zscore')
# df = feature_engineering.get_rolling_means_especificos(df, col='cust_request_qty_zscore')
# df = feature_engineering.get_rolling_stds_especificos(df, col='cust_request_qty_zscore')
# df = feature_engineering.get_rolling_medians_especificos(df, col='cust_request_qty_zscore')
# df = feature_engineering.get_rolling_mins_especificos(df, col='cust_request_qty_zscore')
# df = feature_engineering.get_rolling_maxs_especificos(df, col='cust_request_qty_zscore')
# df = feature_engineering.get_rolling_skewness_especificos(df, col='cust_request_qty_zscore')

##### FE Calendario

In [8]:
df = feature_engineering.generar_ids(df)
df = feature_engineering.get_componentesTemporales(df)
df = feature_engineering.get_anomaliasPoliticas(df)
# df = feature_engineering.descomposicion_serie_temporal(df, col='tn')
df.shape

(31362, 651)

##### FE sobre FE

In [9]:
df = feature_engineering.chatGPT_features_serie(df, "tn")
df = feature_engineering.mes_con_feriado(df)
df.shape

(31362, 680)

##### Variables Exogenas

In [10]:
df = feature_engineering.get_dolar(df)
df = feature_engineering.get_IPC(df)
df['ipc'] = df['ipc'].str.replace(',', '.').astype(float)
df['dolar'] = df['dolar'].str.replace(',', '.').astype(float)
# df.drop(columns=['ds'], inplace=True)
df.fillna(0, inplace=True) ##### EXPERIMENTAR
df = feature_engineering.correlacion_exogenas(df)
df = feature_engineering.get_mes_receso_escolar(df)
df.shape

(31362, 683)

##### EXP 2

-   Es el EXP 1 pero con linear_tree=True

Conclusion: dio horrible.


In [11]:
study, models, results = model_lgb.main_entrenar(df, VERSION="v12")

[I 2025-06-26 12:21:17,525] A new study created in memory with name: no-name-7dc7dd47-144a-413d-ab78-f7f48e5a2e32


Features: ['product_id', 'periodo', 'nacimiento_producto', 'muerte_producto', 'mes_n', 'total_meses', 'producto_nuevo', 'ciclo_de_vida_inicial', 'sku_size', 'stock_final', 'tn', 'plan_precios_cuidados', 'cust_request_qty', 'cust_request_tn', 'cat1_FOODS', 'cat1_HC', 'cat1_PC', 'cat1_REF', 'cat1_nan', 'cat2_ADEREZOS', 'cat2_CABELLO', 'cat2_DENTAL', 'cat2_DEOS', 'cat2_HOGAR', 'cat2_OTROS', 'cat2_PIEL1', 'cat2_PIEL2', 'cat2_PROFESIONAL', 'cat2_ROPA ACONDICIONADOR', 'cat2_ROPA LAVADO', 'cat2_ROPA MANCHAS', 'cat2_SOPAS Y CALDOS', 'cat2_TE', 'cat2_VAJILLA', 'cat2_nan', 'cat3_ACONDICIONADOR', 'cat3_Acond Bebe', 'cat3_Acond Mujer', 'cat3_Aero', 'cat3_Aji Picante', 'cat3_Almidon', 'cat3_Antigrasa', 'cat3_Azul', 'cat3_BARRAS', 'cat3_Baking Bags', 'cat3_Barbacoa', 'cat3_Barra', 'cat3_Baño', 'cat3_Bizcochuelo', 'cat3_CREMA', 'cat3_CUIDADO ESPECIAL', 'cat3_Caldo Cubo', 'cat3_Caldos Granulados', 'cat3_Cara', 'cat3_Cepilllo', 'cat3_Chimichurri', 'cat3_Concentrado', 'cat3_Cremoso', 'cat3_Cristalino', 

[I 2025-06-26 12:21:55,279] Trial 0 finished with value: 56.02267645023521 and parameters: {'max_depth': 9, 'learning_rate': 0.2536999076681772, 'n_estimators': 759, 'min_child_weight': 6, 'subsample': 0.5780093202212182, 'colsample_bytree': 0.5779972601681014, 'reg_alpha': 0.05808361216819946, 'reg_lambda': 0.8661761457749352}. Best is trial 0 with value: 56.02267645023521.


Early stopping, best iteration is:
[5]	valid_0's l1: 47.1485
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[14]	valid_0's l1: 48.4562
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[9]	valid_0's l1: 80.3882
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[18]	valid_0's l1: 34.2821
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[5]	valid_0's l1: 117.186
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[14]	valid_0's l1: 44.9091
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[1]	valid_0's l1: 186.121
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[12]	valid_0's l1: 54.564
Training until validation scores don't improve for 30 rounds
Early stopping, best 

[I 2025-06-26 12:22:26,842] Trial 1 finished with value: 82.47707589755126 and parameters: {'max_depth': 13, 'learning_rate': 0.11114989443094977, 'n_estimators': 118, 'min_child_weight': 10, 'subsample': 0.9162213204002109, 'colsample_bytree': 0.6061695553391381, 'reg_alpha': 0.18182496720710062, 'reg_lambda': 0.18340450985343382}. Best is trial 0 with value: 56.02267645023521.


...

Early stopping, best iteration is:
[2]	valid_0's l1: 172.641
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[12]	valid_0's l1: 43.7607
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[9]	valid_0's l1: 61.7389
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[3]	valid_0's l1: 143.537
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[11]	valid_0's l1: 42.7199
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[1]	valid_0's l1: 179.123
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[14]	valid_0's l1: 37.9855
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[3]	valid_0's l1: 130.184
Training until validation scores don't improve for 30 rounds
Early stopping, best 

[I 2025-06-26 12:23:28,426] Trial 3 finished with value: 83.3036342957619 and parameters: {'max_depth': 11, 'learning_rate': 0.14447746112718687, 'n_estimators': 279, 'min_child_weight': 6, 'subsample': 0.7962072844310213, 'colsample_bytree': 0.5232252063599989, 'reg_alpha': 0.6075448519014384, 'reg_lambda': 0.17052412368729153}. Best is trial 0 with value: 56.02267645023521.


Early stopping, best iteration is:
[2]	valid_0's l1: 143.043
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[13]	valid_0's l1: 14.5723
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[6]	valid_0's l1: 62.0665
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[14]	valid_0's l1: 18.6257
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[14]	valid_0's l1: 14.4794
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[3]	valid_0's l1: 90.5037
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[11]	valid_0's l1: 19.5135
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[6]	valid_0's l1: 41.8273
Training until validation scores don't improve for 30 rounds
Early stopping, best

[I 2025-06-26 12:23:58,443] Trial 4 finished with value: 30.237877249683756 and parameters: {'max_depth': 4, 'learning_rate': 0.2521267904777921, 'n_estimators': 970, 'min_child_weight': 9, 'subsample': 0.6523068845866853, 'colsample_bytree': 0.5488360570031919, 'reg_alpha': 0.6842330265121569, 'reg_lambda': 0.4401524937396013}. Best is trial 4 with value: 30.237877249683756.


Early stopping, best iteration is:
[15]	valid_0's l1: 10.8677

🏆 Mejores parámetros:
max_depth: 4
learning_rate: 0.2521267904777921
n_estimators: 970
min_child_weight: 9
subsample: 0.6523068845866853
colsample_bytree: 0.5488360570031919
reg_alpha: 0.6842330265121569
reg_lambda: 0.4401524937396013
boosting_type: gbdt
objective: regression
metric: mae
verbosity: -1
deterministic: True
force_col_wise: True
linear_tree: True

Entrenando modelo final...
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[3]	valid_0's l1: 83.5421
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[10]	valid_0's l1: 15.8599
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[6]	valid_0's l1: 37.3005
Ensemble MAE Val: 44.8233

🎯 MAE Test Ensemble: 47.1388

Top 5 mejores trials:
1. MAE: 30.2379
2. MAE: 56.0227
3. MAE: 82.4771
4. MAE: 83.3036
5. MAE: 169.4630


In [12]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Supongamos que querés predecir para los períodos 202011 y 202012
periodos_future = [201911, 201912]
df_future = df[df['periodo'].isin(periodos_future)].copy()

# Asegurate de tener las mismas columnas que en entrenamiento
features = [col for col in df.columns if col not in ['target']]

# Predecir
future_preds = model_lgb.predict_with_ensemble(models, df_future, features)

# Si querés evaluar el MAE (solo si tenés target en esos períodos)
if 'target' in df_future:
    mae_future = mean_absolute_error(df_future['target'], future_preds)
    print(f"📊 MAE Future: {mae_future:.4f}")

# Guardar las predicciones
df_future['pred_ensemble'] = future_preds

📊 MAE Future: 73.0768


In [13]:
productos_ok = pd.read_csv("https://storage.googleapis.com/open-courses/austral2025-af91/labo3v/product_id_apredecir201912.txt", sep="\t")
df_future = df_future[df_future['periodo'] == 201912]
df_future = df_future[df_future['product_id'].isin(productos_ok['product_id'].unique())]
df_future = df_future[['product_id', 'pred_ensemble']]
df_future


,product_id,pred_ensemble
30476,20001,1316.054162
30477,20002,1071.481169
30478,20003,660.715471
30479,20004,573.617062
30480,20005,504.629207
...,...,...
31355,21263,45.766781
31357,21265,45.830594
31358,21266,45.831519
31359,21267,45.767863


In [14]:
df[(df['periodo'] == 201912 ) & (df['product_id'].isin([20375]))][['tn']]

,tn
30808,4.38548


In [15]:
len(df_future[df_future['pred_ensemble'] < 10]['product_id'].unique())

0

opcion 1: 
-   menores a 10 tn => promedio 12 meses => 0.291
-   menores a 50 tn => promedio 12 meses => 0.273
-   menores a 81 tn => promedio 12 meses => 0.265
-   menores a 101 tn => promedio 12 meses => 0.275
-   menores a 90 tn => promedio 12 meses => 0.272


In [16]:
# # A los productos < 10 tn les pongo el PROMEDIO ULTIMOS 12 MESES
# productos_negativos = df_future[df_future['pred_ensemble'] < 90]['product_id'].unique()
# df_negativos = feature_engineering.calcular_promedios_12m(productos_negativos, df )
# df_future = df_future.merge(df_negativos, on='product_id', how='left')

# df_future['promedio_12m'] = df_future['promedio_12m'].fillna(df_future['pred_ensemble'])
# df_future



guardo el archivo

In [17]:
# df_future_c = df_future[['product_id', 'promedio_12m']].copy()
# df_future_c.rename(columns={'promedio_12m': 'tn'}, inplace=True)
# df_future_c.to_csv("./outputs/predicciones_exp_02_lgb_fe_v1.csv", index=False, sep=',')